In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
from basemodels.models import Match
import pandas as pd

data = Match.objects.all().values()
data_list = list(data)
df = pd.DataFrame(data_list)
df

,id,startTimestamp,slug,tournament_id,season_id,roundInfo_round,status_type,homeTeam_id,awayTeam_id,homeScore_current,homeScore_period1,homeScore_period2,homeScore_normaltime,awayScore_current,awayScore_period1,awayScore_period2,awayScore_normaltime
0,11907436,2024-10-21 12:00:00+00:00,djurgardens-if-hammarby-if,40,57284,27,finished,1758,1759,2,2,0,2,0,0,0,0
1,11907437,2024-10-21 17:10:00+00:00,aik-ifk-goteborg,40,57284,27,finished,1761,1764,1,0,1,1,2,1,1,2
2,11907438,2024-10-21 17:00:00+00:00,if-brommapojkarna-ifk-norrkoping,40,57284,27,finished,1769,1787,1,1,0,1,1,1,0,1
3,11907440,2024-10-20 14:30:00+00:00,ifk-varnamo-gais-goteborg,40,57284,27,finished,1786,1842,0,0,0,0,0,0,0,0
4,11907442,2024-10-20 12:00:00+00:00,kalmar-ff-mjallby-aif,40,57284,27,finished,1891,1783,0,0,0,0,3,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,12991468,2024-10-20 20:30:00+00:00,sol-de-america-cerro-porteno,11541,63769,16,finished,36439,5991,0,0,0,0,1,1,0,1
478,12991470,2024-10-20 23:00:00+00:00,olimpia-libertad,11541,63769,16,finished,5996,5997,0,0,0,0,1,0,1,1
479,12992167,2024-10-20 17:15:00+00:00,ksc-lokeren-temse-rfc-seraing,9,61412,8,finished,371484,204712,0,0,0,0,1,1,0,1
480,13008795,2024-10-22 23:30:00+00:00,atlanta-united-cf-montreal,242,57317,0,notstarted,22006,243211,0,0,0,0,0,0,0,0


In [10]:
import pandas as pd



# Puan durumu hesaplamaya başla
def puan_durumu(df,  season_id):
    # İlgili sezon ve turnuvadaki maçları filtrele
    df = df[ (df['season_id'] == season_id) & (df['status_type'] == 'finished')]

    # Tüm takımları bul
    teams = pd.concat([df['homeTeam_id'], df['awayTeam_id']]).unique()

    # Puan durumu için gerekli sütunları oluştur
    puan_durumu = pd.DataFrame(teams, columns=['team_id'])
    puan_durumu['W'] = 0  # Galibiyet
    puan_durumu['D'] = 0  # Beraberlik
    puan_durumu['L'] = 0  # Mağlubiyet
    puan_durumu['GF'] = 0  # Atılan Goller
    puan_durumu['GA'] = 0  # Yenen Goller
    puan_durumu['Pts'] = 0  # Puan

    # Her maç için takımların performansını güncelle
    for _, row in df.iterrows():
        home_team = row['homeTeam_id']
        away_team = row['awayTeam_id']
        home_score = row['homeScore_current']
        away_score = row['awayScore_current']

        # Atılan ve yenilen golleri güncelle
        puan_durumu.loc[puan_durumu['team_id'] == home_team, 'GF'] += home_score
        puan_durumu.loc[puan_durumu['team_id'] == home_team, 'GA'] += away_score
        puan_durumu.loc[puan_durumu['team_id'] == away_team, 'GF'] += away_score
        puan_durumu.loc[puan_durumu['team_id'] == away_team, 'GA'] += home_score

        # Maç sonucuna göre puan, galibiyet, beraberlik, mağlubiyet güncelle
        if home_score > away_score:
            # Ev sahibi kazandı
            puan_durumu.loc[puan_durumu['team_id'] == home_team, 'W'] += 1
            puan_durumu.loc[puan_durumu['team_id'] == home_team, 'Pts'] += 3
            puan_durumu.loc[puan_durumu['team_id'] == away_team, 'L'] += 1
        elif home_score < away_score:
            # Deplasman kazandı
            puan_durumu.loc[puan_durumu['team_id'] == away_team, 'W'] += 1
            puan_durumu.loc[puan_durumu['team_id'] == away_team, 'Pts'] += 3
            puan_durumu.loc[puan_durumu['team_id'] == home_team, 'L'] += 1
        else:
            # Beraberlik
            puan_durumu.loc[puan_durumu['team_id'] == home_team, 'D'] += 1
            puan_durumu.loc[puan_durumu['team_id'] == home_team, 'Pts'] += 1
            puan_durumu.loc[puan_durumu['team_id'] == away_team, 'D'] += 1
            puan_durumu.loc[puan_durumu['team_id'] == away_team, 'Pts'] += 1

    # Puan durumunu puanlara göre sıralama
    puan_durumu = puan_durumu.sort_values(by=['Pts', 'GF', 'GA'], ascending=[False, False, True])
    puan_durumu["season_id"]=season_id
    
    return puan_durumu

# Sezon ve turnuva için puan durumu
puan_tablosu = puan_durumu(df, season_id=	61667)

puan_tablosu 


,team_id,W,D,L,GF,GA,Pts,season_id
4,2967,1,0,0,3,1,3,61667
2,2970,1,0,0,2,0,3,61667
1,2974,1,0,0,2,1,3,61667
3,46160,0,1,0,0,0,1,61667
7,2966,0,1,0,0,0,1,61667
5,2954,0,0,1,1,2,0,61667
0,46168,0,0,1,1,3,0,61667
6,2994,0,0,1,0,2,0,61667


In [2]:
#verileri oku
from basemodels.models import FikstureModelData
import pandas as pd

data = FikstureModelData.objects.all().values()
data_list = list(data)
df = pd.DataFrame(data_list)
df_exploded = df.explode('data')
expanded_data = pd.json_normalize(df_exploded['data'])
expanded_data['tarih'] = list(df_exploded['tarih'])
expanded_data

,customId,winnerCode,hasGlobalHighlights,hasEventPlayerStatistics,hasEventPlayerHeatMap,detailId,crowdsourcingDataDisplayEnabled,id,startTimestamp,slug,...,homeTeam.gender,awayTeam.gender,homeScore.aggregated,awayScore.aggregated,homeScore.penalties,awayScore.penalties,tournament.uniqueTournament.primaryColorHex,tournament.uniqueTournament.secondaryColorHex,coverage,tarih
0,dsr,2.0,True,True,True,1.0,False,12437039,1729429200,manchester-city-wolverhampton,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
1,NsU,1.0,True,True,True,1.0,False,12437032,1729438200,liverpool-chelsea,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
2,hso,1.0,True,True,True,1.0,False,12437036,1729537200,nottingham-forest-crystal-palace,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
3,tgbsBgb,1.0,True,True,True,1.0,False,12437847,1729425600,mallorca-rayo-vallecano,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
4,LgbsVgb,1.0,True,True,True,1.0,False,12437835,1729433700,leganes-atletico-madrid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,fYbsmYb,NaN,False,NaN,NaN,NaN,False,12872190,1729622700,connahs-quay-nomads-aberystwyth-town,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-22
550,lQrsjlLb,NaN,False,NaN,NaN,NaN,False,12464047,1729638000,ecuador-u17-new-zealand-u17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-22
551,fQrsXlpd,NaN,False,NaN,NaN,NaN,False,12464084,1729638000,dominican-republic-u17-nigeria-u17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-22
552,wlpskEu,NaN,False,NaN,NaN,NaN,False,12464043,1729627200,colombia-u17-spain-u17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-22


In [5]:
expanded_data.to_excel("FullData.xlsx")

In [4]:
list(expanded_data.columns)

['customId',
 'winnerCode',
 'hasGlobalHighlights',
 'hasEventPlayerStatistics',
 'hasEventPlayerHeatMap',
 'detailId',
 'crowdsourcingDataDisplayEnabled',
 'id',
 'startTimestamp',
 'slug',
 'finalResultOnly',
 'feedLocked',
 'isEditor',
 'tournament.name',
 'tournament.slug',
 'tournament.category.name',
 'tournament.category.slug',
 'tournament.category.sport.name',
 'tournament.category.sport.slug',
 'tournament.category.sport.id',
 'tournament.category.id',
 'tournament.category.country.alpha2',
 'tournament.category.country.alpha3',
 'tournament.category.country.name',
 'tournament.category.country.slug',
 'tournament.category.flag',
 'tournament.category.alpha2',
 'tournament.uniqueTournament.name',
 'tournament.uniqueTournament.slug',
 'tournament.uniqueTournament.category.name',
 'tournament.uniqueTournament.category.slug',
 'tournament.uniqueTournament.category.sport.name',
 'tournament.uniqueTournament.category.sport.slug',
 'tournament.uniqueTournament.category.sport.id',
 

In [8]:
model_cols = ['id','startTimestamp','slug','tournament.uniqueTournament.id','season.id',
                'roundInfo.round','status.type','homeTeam.id','awayTeam.id',
                'homeScore.current','homeScore.display','homeScore.period1','homeScore.period2','homeScore.normaltime',
                'awayScore.current','awayScore.display','awayScore.period1','awayScore.period2','awayScore.normaltime',
                ]

new_cols = ['id','date','slug','tournament_uniqueTournament_id','season_id',
                'roundInfo_round','status_type','homeTeam_id','awayTeam_id',
                'homeScore_current','homeScore_display','homeScore_period1','homeScore_period2','homeScore_normaltime',
                'awayScore_current','awayScore_display','awayScore_period1','awayScore_period2','awayScore_normaltime',
                ]


df1 = expanded_data[model_cols]
df1.columns = new_cols
df1

,id,date,slug,tournament_uniqueTournament_id,season_id,roundInfo_round,status_type,homeTeam_id,awayTeam_id,homeScore_current,homeScore_display,homeScore_period1,homeScore_period2,homeScore_normaltime,awayScore_current,awayScore_display,awayScore_period1,awayScore_period2,awayScore_normaltime
0,12437039,1729429200,manchester-city-wolverhampton,17,61627,8.0,finished,3,17,1.0,1.0,1.0,0.0,1.0,2.0,2.0,1.0,1.0,2.0
1,12437032,1729438200,liverpool-chelsea,17,61627,8.0,finished,44,38,2.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0
2,12437036,1729537200,nottingham-forest-crystal-palace,17,61627,8.0,finished,14,7,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,12437847,1729425600,mallorca-rayo-vallecano,8,61643,10.0,finished,2826,2818,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,12437835,1729433700,leganes-atletico-madrid,8,61643,10.0,finished,2836,2845,3.0,3.0,0.0,3.0,3.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,12872190,1729622700,connahs-quay-nomads-aberystwyth-town,367,61686,27.0,notstarted,4912,4905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550,12464047,1729638000,ecuador-u17-new-zealand-u17,827,61934,3.0,notstarted,215559,44561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551,12464084,1729638000,dominican-republic-u17-nigeria-u17,827,61934,3.0,notstarted,44555,413097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
552,12464043,1729627200,colombia-u17-spain-u17,827,61934,3.0,notstarted,48960,38071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
from basemodels.models import Tournament, Season, Team,Match
from basemodels.models import FikstureModelData
from datetime import datetime, timedelta

class SofascoreAPI:
    def __init__(self):
        pass

    def fetch_data(self):
        data = FikstureModelData.objects.all().values()
        data_list = list(data)
        df = pd.DataFrame(data_list)
        df_exploded = df.explode('data')
        expanded_data = pd.json_normalize(df_exploded['data'])
        expanded_data['tarih'] = list(df_exploded['tarih'])
        veri = expanded_data.copy()
        veri.columns = [item.replace(".", "_") for item in veri.columns]
        return veri

    def tournament_data(self):
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        # Aynı uniqueTournament_id'ye sahip olanları tekrar kontrol et
        verim = verim.drop_duplicates("tournament_uniqueTournament_id")
        
        cols = [
            "tournament_uniqueTournament_id", "tournament_name", "tournament_slug", "tournament_category_name",
            "tournament_category_slug", "tournament_uniqueTournament_category_id", 
            "tournament_uniqueTournament_hasEventPlayerStatistics", "tournament_id", "tournament_isGroup", 
            "tournament_uniqueTournament_hasPerformanceGraphFeature"
        ]
        
        for col in cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        verim = verim[cols]

        for _, row in verim.iterrows():
            tournament_id = row['tournament_uniqueTournament_id']
            
            if pd.isna(tournament_id):
                print(f"Geçersiz tournament_uniqueTournament_id: {tournament_id}, atlanıyor.")
                continue
        
            if not Tournament.objects.filter(tournament_id=tournament_id).exists():
                Tournament.objects.create(
                    tournament_id=row['tournament_uniqueTournament_id'],
                    tournament_name=row['tournament_name'],
                    tournament_slug=row['tournament_slug'],
                    tournament_category_name=row['tournament_category_name'],
                    tournament_category_slug=row['tournament_category_slug'],
                    tournament_uniqueTournament_category_id=row['tournament_uniqueTournament_category_id'],
                    tournament_model_id=row['tournament_id'],
                    tournament_isGroup=row['tournament_isGroup'] if pd.notna(row['tournament_isGroup']) else False,
                    tournament_uniqueTournament_hasPerformanceGraphFeature=row['tournament_uniqueTournament_hasPerformanceGraphFeature'] if pd.notna(row['tournament_uniqueTournament_hasPerformanceGraphFeature']) else False
                )
                print(f"Tournament ID {tournament_id} başarıyla eklendi.")
            else:
                print(f"Tournament ID {tournament_id} zaten mevcut, eklenmedi.")

    def season_data(self):
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        verim = verim.drop_duplicates("season_id")
        
        season_cols = ["season_id", "season_name", "season_year","tournament_uniqueTournament_id"]

        for col in season_cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        verim = verim[season_cols]

        for _, row in verim.iterrows():
            season_id = row['season_id']
            tournament_id = row.get('tournament_uniqueTournament_id')
    
            if pd.isna(season_id) or pd.isna(tournament_id):
                print(f"Geçersiz season_id veya tournament_id: Season ID={season_id}, Tournament ID={tournament_id}. Atlanıyor.")
                continue
    
            if not Season.objects.filter(season_id=season_id).exists():
                tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
                
                if tournament_instance:
                    Season.objects.create(
                        season_id=row['season_id'],
                        season_name=row['season_name'],
                        season_year=row['season_year'],
                        tournament=tournament_instance  # Doğru sütun adı burada kullanılıyor
                    )
                    print(f"Season ID {season_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} bulunamadı, Season ID {season_id} eklenemedi.")
            else:
                print(f"Season ID {season_id} zaten mevcut, eklenmedi.")
    
   
    def team_data(self):

        data = self.fetch_data()
        homecols = [
                    "homeTeam_id", "homeTeam_name", "homeTeam_slug", "homeTeam_shortName",
                    "homeTeam_nameCode", "homeTeam_national", "tournament_uniqueTournament_id", "season_id"
                ]
        
        awaycols = [
                    "awayTeam_id", "awayTeam_name", "awayTeam_slug", "awayTeam_shortName",
                    "awayTeam_nameCode", "awayTeam_national", "tournament_uniqueTournament_id", "season_id"
                ]
        
        # Home takımı için veri seçimi ve kolon isimlerinin ayarlanması
        homedata = data[homecols]
        homedata.columns = ["team_id", "team_name", "team_slug", "team_shortName",
                                       "team_nameCode", "team_national", "tournament_id", "season_id"]
        
        # Away takımı için veri seçimi ve kolon isimlerinin ayarlanması
        awaydata = data[awaycols]
        awaydata.columns  = ["team_id", "team_name", "team_slug", "team_shortName",
                                       "team_nameCode", "team_national", "tournament_id", "season_id"]
        
        # Home ve Away takımları birleştirme
        teamdata = pd.concat([homedata, awaydata])
        teamdata = teamdata.drop_duplicates(subset=["team_id", "tournament_id", "season_id"])
       
        for _, row in teamdata.iterrows():
            team_id = row['team_id']
            tournament_id = row['tournament_id']
            season_id = row['season_id']
            
            # Geçersiz veya eksik id'leri atla
            if pd.isna(team_id) or pd.isna(tournament_id) or pd.isna(season_id):
                print(f"Geçersiz değer: Team ID={team_id}, Tournament ID={tournament_id}, Season ID={season_id}. Atlanıyor.")
                continue

            # İlgili tournament ve season nesnelerini getir
            tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
            season_instance = Season.objects.filter(season_id=season_id).first()

            if tournament_instance and season_instance:
                # Takımın veritabanında var olup olmadığını kontrol et, yoksa oluştur
                if not Team.objects.filter(team_id=team_id, tournament=tournament_instance, season=season_instance).exists():
                    Team.objects.create(
                        team_id=team_id,
                        team_name=row['team_name'],
                        team_slug=row['team_slug'],
                        team_shortName=row['team_shortName'],
                        team_nameCode=row['team_nameCode'],
                        team_national=row['team_national'],
                        tournament=tournament_instance,  # ForeignKey olan alanlar
                        season=season_instance           # ForeignKey olan alanlar
                    )
                    print(f"Team ID {team_id} başarıyla eklendi.")
                else:
                    print(f"Team ID {team_id} zaten mevcut, eklenmedi.")
            else:
                print(f"Tournament ID {tournament_id} veya Season ID {season_id} bulunamadı.")

    def match_data(self):
        # Fetch data from the API or existing model
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Lütfen önce 'process_data()' metodunu çalıştırın.")
        
     
     
        new_cols = ['id', 'startTimestamp', 'slug', 'tournament_uniqueTournament_id', 'season_id',
                    'roundInfo_round', 'status_type', 'homeTeam_id', 'awayTeam_id',
                    'homeScore_current', 'homeScore_period1', 'homeScore_period2', 'homeScore_normaltime',
                    'awayScore_current', 'awayScore_period1', 'awayScore_period2', 'awayScore_normaltime']
        
        df1 = verim[new_cols] 
        df1['date'] = pd.to_datetime(df1['startTimestamp'], unit='s') + timedelta(hours=3)
        df2 = df1.drop("startTimestamp", axis=1)
        df3 = df2.fillna(0)
        for _, row in df3.iterrows():
            match_id = row['id']
            tournament_id = row['tournament_uniqueTournament_id']
            season_id = row['season_id']
            home_team_id = row['homeTeam_id']
            away_team_id = row['awayTeam_id']
            
            # Geçersiz veya eksik id'leri atla
            if pd.isna(match_id) or pd.isna(tournament_id) or pd.isna(season_id) or pd.isna(home_team_id) or pd.isna(away_team_id):
                print(f"Geçersiz değer: Match ID={match_id}, Tournament ID={tournament_id}, Season ID={season_id}, Home Team ID={home_team_id}, Away Team ID={away_team_id}. Atlanıyor.")
                continue

            # İlgili foreign key nesnelerini getir
            tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
            season_instance = Season.objects.filter(season_id=season_id).first()
            home_team_instance = Team.objects.filter(team_id=home_team_id).first()
            away_team_instance = Team.objects.filter(team_id=away_team_id).first()

            if tournament_instance and season_instance and home_team_instance and away_team_instance:
                # Match'in veritabanında olup olmadığını kontrol et, yoksa oluştur
                if not Match.objects.filter(id=match_id).exists():
                    Match.objects.create(
                        id=match_id,
                        startTimestamp=row['date'],  # DateTimeField olarak datetime verisi
                        slug=row['slug'],
                        tournament=tournament_instance,
                        season=season_instance,
                        homeTeam=home_team_instance,
                        awayTeam=away_team_instance,
                        roundInfo_round=row['roundInfo_round'],
                        homeScore_current=row['homeScore_current'],
                        homeScore_period1=row['homeScore_period1'],
                        homeScore_period2=row['homeScore_period2'],
                        homeScore_normaltime=row['homeScore_normaltime'],
                        awayScore_current=row['awayScore_current'],
                        awayScore_period1=row['awayScore_period1'],
                        awayScore_period2=row['awayScore_period2'],
                        awayScore_normaltime=row['awayScore_normaltime'],
                        status_type = row['status_type']
                    )
                    print(f"Match ID {match_id} başarıyla eklendi.")
                else:
                    print(f"Match ID {match_id} zaten mevcut, eklenmedi.")
            else:
                print(f"Tournament ID {tournament_id}, Season ID {season_id}, Home Team ID {home_team_id}, Away Team ID {away_team_id} bulunamadı.")





# SofascoreAPI sınıfını kullanarak team_data'yı veritabanına kaydetmek için:
api = SofascoreAPI()
teamdata = api.fetch_data()  # Veriyi getirin (API'den ya da dosyadan)
#api.tournament_data()
#api.season_data()
#api.team_data()
api.match_data()
print("İşlemler bitti")


C:\Users\ahmet.yildirir\AppData\Local\Temp\ipykernel_12468\1173372429.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['date'] = pd.to_datetime(df1['startTimestamp'], unit='s') + timedelta(hours=3)


Match ID 12437039 zaten mevcut, eklenmedi.
Match ID 12437032 zaten mevcut, eklenmedi.
Match ID 12437036 zaten mevcut, eklenmedi.
Match ID 12437847 zaten mevcut, eklenmedi.
Match ID 12437835 zaten mevcut, eklenmedi.
Match ID 12437852 zaten mevcut, eklenmedi.
Match ID 12437839 zaten mevcut, eklenmedi.
Match ID 12437854 zaten mevcut, eklenmedi.
Match ID 12499331 zaten mevcut, eklenmedi.
Match ID 12499335 zaten mevcut, eklenmedi.
Match ID 12499325 zaten mevcut, eklenmedi.
Match ID 12499334 zaten mevcut, eklenmedi.
Match ID 12499329 zaten mevcut, eklenmedi.
Match ID 12499710 zaten mevcut, eklenmedi.
Match ID 12499700 zaten mevcut, eklenmedi.
Match ID 12451038 zaten mevcut, eklenmedi.
Match ID 12451034 zaten mevcut, eklenmedi.
Match ID 12451040 zaten mevcut, eklenmedi.
Match ID 12451036 zaten mevcut, eklenmedi.
Match ID 12451042 zaten mevcut, eklenmedi.
Match ID 12448385 zaten mevcut, eklenmedi.
Match ID 12448386 zaten mevcut, eklenmedi.
Match ID 12448387 zaten mevcut, eklenmedi.
Match ID 12

C:\Users\ahmet.yildirir\AppData\Local\Programs\Python\Python312\Lib\site-packages\django\db\models\fields\__init__.py:1595: RuntimeWarning: DateTimeField Match.startTimestamp received a naive datetime (2024-10-22 14:00:00) while time zone support is active.
  warnings.warn(


Match ID 12792310 başarıyla eklendi.


C:\Users\ahmet.yildirir\AppData\Local\Programs\Python\Python312\Lib\site-packages\django\db\models\fields\__init__.py:1595: RuntimeWarning: DateTimeField Match.startTimestamp received a naive datetime (2024-10-22 15:00:00) while time zone support is active.
  warnings.warn(


...

Match ID 12855335 başarıyla eklendi.
Match ID 12526428 başarıyla eklendi.
Match ID 12526425 başarıyla eklendi.
Match ID 12526424 başarıyla eklendi.
Match ID 12526423 başarıyla eklendi.
Match ID 12526421 başarıyla eklendi.
Match ID 12526418 başarıyla eklendi.
Match ID 11990379 başarıyla eklendi.
Match ID 11990378 başarıyla eklendi.
Match ID 13016256 başarıyla eklendi.
Match ID 12525935 başarıyla eklendi.
Match ID 12525937 başarıyla eklendi.
Match ID 12525939 başarıyla eklendi.
Match ID 12525938 başarıyla eklendi.
Match ID 12525934 başarıyla eklendi.
Match ID 12460022 başarıyla eklendi.
Match ID 12460019 başarıyla eklendi.
Match ID 12460021 başarıyla eklendi.
Match ID 12460026 başarıyla eklendi.
Match ID 12460029 başarıyla eklendi.
Match ID 12460024 başarıyla eklendi.
Match ID 12460016 başarıyla eklendi.
Match ID 12460020 başarıyla eklendi.
Match ID 12460023 başarıyla eklendi.
Match ID 12819281 zaten mevcut, eklenmedi.


C:\Users\ahmet.yildirir\AppData\Local\Programs\Python\Python312\Lib\site-packages\django\db\models\fields\__init__.py:1595: RuntimeWarning: DateTimeField Match.startTimestamp received a naive datetime (2024-10-23 11:30:00) while time zone support is active.
  warnings.warn(


Match ID 12650999 başarıyla eklendi.
Match ID 12651008 başarıyla eklendi.
Match ID 12545562 zaten mevcut, eklenmedi.
Match ID 12545220 zaten mevcut, eklenmedi.
Match ID 12545217 zaten mevcut, eklenmedi.
Match ID 12598366 zaten mevcut, eklenmedi.
Match ID 12598368 zaten mevcut, eklenmedi.
Match ID 12817284 başarıyla eklendi.
Match ID 12852980 zaten mevcut, eklenmedi.
Match ID 12855815 başarıyla eklendi.
Match ID 12653643 zaten mevcut, eklenmedi.
Match ID 12853213 başarıyla eklendi.
Match ID 12969180 başarıyla eklendi.
Match ID 12969186 başarıyla eklendi.
Match ID 12969183 başarıyla eklendi.
Match ID 12969184 başarıyla eklendi.
Match ID 12790838 başarıyla eklendi.
Match ID 11918303 zaten mevcut, eklenmedi.
Match ID 11918297 zaten mevcut, eklenmedi.
Match ID 11918298 zaten mevcut, eklenmedi.
Match ID 11918299 zaten mevcut, eklenmedi.
Match ID 11918300 zaten mevcut, eklenmedi.
Match ID 11918301 zaten mevcut, eklenmedi.
Match ID 11918302 zaten mevcut, eklenmedi.
Match ID 11918304 zaten mevc

C:\Users\ahmet.yildirir\AppData\Local\Programs\Python\Python312\Lib\site-packages\django\db\models\fields\__init__.py:1595: RuntimeWarning: DateTimeField Match.startTimestamp received a naive datetime (2024-10-22 23:00:00) while time zone support is active.
  warnings.warn(


Match ID 12464043 başarıyla eklendi.
Match ID 12464038 başarıyla eklendi.
İşlemler bitti


In [8]:
Seasonx = Season.objects.all().values()
Seasonx = list(Seasonx)
Seasonx = pd.DataFrame(Seasonx)
Seasonx

,season_id,season_name,season_year,tournament_id
0,57284,Allsvenskan 2024,2024,40
1,57318,USL League One 2024,2024,13362
2,57322,Eliteserien 2024,2024,20
3,57356,1st Division 2024,2024,22
4,57359,"Toppserien, Women 2024",2024,201
...,...,...,...,...
99,65687,Primera Division Women 24/25,24/25,1127
100,65961,Indian Super League 24/25,24/25,1900
101,66456,UEFA Youth League 24/25,24/25,2132
102,66662,Super League 2 24/25,24/25,186


In [9]:
Teamdata = Team.objects.all().values()
Teamdata = list(Teamdata)
Teamdata= pd.DataFrame(Teamdata)
Teamdata

,team_id,team_name,team_slug,team_shortName,team_nameCode,team_national,tournament_id,season_id
0,3,Wolverhampton,wolverhampton,Wolves,WOL,False,17,61627
1,7,Crystal Palace,crystal-palace,Crystal Palace,CRY,False,17,61627
2,14,Nottingham Forest,nottingham-forest,Forest,NFO,False,17,61627
3,17,Manchester City,manchester-city,Man City,MCI,False,17,61627
4,38,Chelsea,chelsea,Chelsea,CHE,False,17,61627
...,...,...,...,...,...,...,...,...
654,826802,Fiorentina U20,acf-fiorentina-u20-u20,Fiorentina U20,ACF,False,2292,64432
655,826810,Udinese U20,udinese-calcio-u20-u20,Udinese U20,UDI,False,2292,64432
656,826812,Genoa U20,genoa-cfc-u20-u20,Genoa U20,GEN,False,2292,64432
657,826817,Sampdoria U20,sampdoria-genoa-u20-u20,Sampdoria U20,SAM,False,2292,64432


In [15]:
takım = Teamdata[Teamdata["team_id"]==3052]
takım

,team_id,team_name,team_slug,team_shortName,team_nameCode,team_national,tournament_id,season_id
200,3052,Fenerbahçe,fenerbahce,Fenerbahçe,FEN,False,52,63814


In [17]:
from basemodels.models import Tournament, Season, Team
Tournamentx = Tournament.objects.all().values()
Tournamentx = list(Tournamentx)
Tournamentx= pd.DataFrame(Tournamentx)
Tournamentx

,tournament_id,tournament_name,tournament_slug,tournament_category_name,tournament_category_slug,tournament_uniqueTournament_category_id,tournament_uniqueTournament_hasEventPlayerStatistics,tournament_model_id,tournament_isGroup,tournament_uniqueTournament_hasPerformanceGraphFeature
0,8,LaLiga,laliga,Spain,spain,32,False,36,False,False
1,9,Challenger Pro League,challenger-pro-league,Belgium,belgium,33,False,692,False,False
2,17,Premier League,premier-league,England,england,1,False,1,False,False
3,18,Championship,championship,England,england,1,False,2,False,False
4,20,Eliteserien,eliteserien,Norway,norway,5,False,5,False,False
...,...,...,...,...,...,...,...,...,...,...
99,11621,"Liga MX, Apertura",liga-mx-apertura,Mexico,mexico,12,False,28,False,False
100,11653,Primera División,primera-division,Chile,chile,49,False,64475,False,False
101,13362,"USL, League One",usl-league-one,USA,usa,26,False,71366,False,False
102,14688,Super Cup,super-cup,Egypt,egypt,305,False,22348,False,False


In [13]:
result = pd.merge(Teamdata,Seasonx , on="season_id")
result

,team_id,team_name,team_slug,team_shortName,team_nameCode,team_national,tournament_id_x,season_id,season_name,season_year,tournament_id_y
0,3,Wolverhampton,wolverhampton,Wolves,WOL,False,17,61627,Premier League 24/25,24/25,17
1,7,Crystal Palace,crystal-palace,Crystal Palace,CRY,False,17,61627,Premier League 24/25,24/25,17
2,14,Nottingham Forest,nottingham-forest,Forest,NFO,False,17,61627,Premier League 24/25,24/25,17
3,17,Manchester City,manchester-city,Man City,MCI,False,17,61627,Premier League 24/25,24/25,17
4,38,Chelsea,chelsea,Chelsea,CHE,False,17,61627,Premier League 24/25,24/25,17
...,...,...,...,...,...,...,...,...,...,...,...
654,405294,Central Valley Fuego,central-valley-fuego,CV Fuego,FUE,False,13362,57318,USL League One 2024,2024,13362
655,826802,Fiorentina U20,acf-fiorentina-u20-u20,Fiorentina U20,ACF,False,2292,64432,Primavera 1 24/25,24/25,2292
656,826810,Udinese U20,udinese-calcio-u20-u20,Udinese U20,UDI,False,2292,64432,Primavera 1 24/25,24/25,2292
657,826812,Genoa U20,genoa-cfc-u20-u20,Genoa U20,GEN,False,2292,64432,Primavera 1 24/25,24/25,2292
